# Home Inventory Demo

This Demo uses the Google Cloud Vision API to create a home inventory, using casually collected photos of a home.

In [57]:
# Uncomment and run this to install dependencies.
# pip install --upgrade google-cloud-vision

In [58]:
import csv

from google.cloud import vision_v1

from util import helpers, data_classes

In [59]:
# Instantiates a client
client = vision_v1.ImageAnnotatorClient()

In [60]:
gs_uris = helpers.get_image_uris("mikebernico-sandbox-sf-home-inventory-demo")

In [61]:
home_objects = [] # list to hold home objects

for uri in gs_uris:
    request = helpers.build_request(uri)
    response = client.annotate_image(request)
    
    if response.full_text_annotation.pages:
        width = response.full_text_annotation.pages[0].width # image width, needed to normalize text_annotations
        height = response.full_text_annotation.pages[0].height # image height, needed to normalize text_annotations
    
    # Parse text_annotations
    text_annotations = []
    for a in response.text_annotations:
        clean_text = a.description.replace('\n',' ')
        new_text_annotation = data_classes.text_annotation(clean_text, height, width)
        new_text_annotation.update_location(a.bounding_poly.vertices[0], a.bounding_poly.vertices[3])
        text_annotations.append(new_text_annotation)
    
        # Parse object responses
    for r in response.localized_object_annotations:
        new_home_object = data_classes.home_object(r.name, r.score)
        new_home_object.update_location(r.bounding_poly.normalized_vertices[0], r.bounding_poly.normalized_vertices[3])      
        home_objects.append(new_home_object)
    
        # Append intersecting text annotations onto home objects
        for t in text_annotations:
            iou = helpers.bb_intersection_over_union(new_home_object.location, t.location)
            if iou > 0.5:
                new_home_object.object_text += " " + t.text

In [62]:
with open('inventory.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Item Name', 'Score', 'Object Text'])
    for item in home_objects:
        filewriter.writerow([item.name, item.score, item.object_text])
        